In [2]:
import json
import os
import requests

# input_dir = f"{destination_base}/2013"

In [5]:
def fetchContentAbstract(url):
    headers = {
        'X-ELS-APIKey': API_KEY,
        'Accept': 'application/json'
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()

        creator = data['abstracts-retrieval-response']['coredata'].get('dc:creator', None)
        if creator is not None:
            creator = data['abstracts-retrieval-response']['coredata']['dc:creator']['author']

        source_id = data['abstracts-retrieval-response']['coredata'].get('source-id', None)
        
        return creator, source_id
    else:
        print(f"Error: {response.status_code}")

In [ ]:
input_dir = f"{destination_base}/2013"

files = [f"{input_dir}/{file}" for file in os.listdir(input_dir) if file.endswith(".json")]

for file in files:
    with open(file, 'r') as f:
        data = json.load(f)
        
        for paper in data:
            print("Fetching content abstract for", paper['prism:doi'])

            author, source_id = fetchContentAbstract(paper['prism:url'])
            
            if author is not None:
                paper['author'] = author
            if source_id is not None:
                paper['source-id'] = source_id

        # with open(f"{os.path.splitext(file)[0]}_fetchContentAbstract.json", 'w') as f:
        #     json.dump(data, f, indent=4)
        
        break

In [3]:
import os
import json
import requests
import time
import random
import logging
from concurrent.futures import ThreadPoolExecutor, as_completed
import tqdm

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s: %(message)s',
    handlers=[
        logging.FileHandler('content_abstract_fetch.log', mode='w'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

In [4]:
class RateLimitHandler:
    def __init__(self, api_key, max_retries=3):
        self.api_key = api_key
        self.max_retries = max_retries
        self.session = requests.Session()

    def fetch_with_retry(self, url, doi):
        headers = {
            'X-ELS-APIKey': self.api_key,
            'Accept': 'application/json'
        }

        for attempt in range(self.max_retries):
            try:
                response = self.session.get(url, headers=headers, timeout=10)
                
                if response.status_code == 429:
                    # Exponential backoff with jitter
                    wait_time = min(60, (2 ** attempt) + random.uniform(0, 1))
                    logger.warning(f"Rate limited. Waiting {wait_time} seconds for DOI {doi}...")
                    time.sleep(wait_time)
                    continue
                
                if response.status_code != 200:
                    logger.error(f"Unexpected status code {response.status_code} for DOI {doi}")
                    return None
                
                return response.json()
            
            except requests.exceptions.RequestException as e:
                logger.error(f"Error fetching {url} for DOI {doi} (Attempt {attempt + 1}): {e}")
                
                # If it's the last attempt, return None
                if attempt == self.max_retries - 1:
                    return None
                
                # Wait a bit before retrying
                time.sleep(1)
        
        return None

In [5]:
def fetchContentAbstract(paper, rate_limiter):
    try:
        doi = paper['prism:doi']
        url = paper['prism:url']
        
        data = rate_limiter.fetch_with_retry(url, doi)
        
        if data is None:
            logger.warning(f"No data retrieved for DOI: {doi}")
            return None
        
        try:
            # More robust data extraction with extensive logging
            creator = data.get('abstracts-retrieval-response', {}) \
                        .get('coredata', {}) \
                        .get('dc:creator', None)
            
            if creator is not None:
                # Handle different possible structures of creator
                if isinstance(creator, dict):
                    creator = creator.get('author')
                elif isinstance(creator, list):
                    creator = [a.get('author', a) for a in creator]
            
            source_id = data.get('abstracts-retrieval-response', {}) \
                           .get('coredata', {}) \
                           .get('source-id', None)
            
            return {
                'doi': doi,
                'author': creator,
                'source_id': source_id
            }
        
        except Exception as parse_error:
            logger.error(f"Error parsing data for DOI {doi}: {parse_error}")
            logger.error(f"Problematic data structure: {data}")
            return None
    
    except Exception as e:
        logger.error(f"Processing error for DOI {doi}: {e}")
        return None

In [6]:
def process_file(file_path, api_key, max_workers=3):
    # Read input file
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    # Initialize rate limiter
    rate_limiter = RateLimitHandler(api_key)
    
    # Track processed papers
    processed_data = data.copy()
    
    logger.info(f"Processing {len(data)} papers from {file_path}")
    
    # Track successful and failed retrievals
    successful_retrievals = 0
    failed_retrievals = 0
    
    # Use ThreadPoolExecutor for concurrent processing
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Progress bar
        progress_bar = tqdm.tqdm(total=len(data), desc="Fetching Abstracts", unit="paper")
        
        # Submit all tasks
        futures = {
            executor.submit(fetchContentAbstract, paper, rate_limiter): paper 
            for paper in data
        }
        
        # Process results as they complete
        for future in as_completed(futures):
            original_paper = futures[future]
            result = future.result()
            
            # Update progress
            progress_bar.update(1)
            
            if result:
                # Find and update the corresponding paper
                for updated_paper in processed_data:
                    if updated_paper['prism:doi'] == result['doi']:
                        if result['author'] is not None:
                            updated_paper['author'] = result['author']
                        if result['source_id'] is not None:
                            updated_paper['source-id'] = result['source_id']
                        successful_retrievals += 1
                        break
            else:
                failed_retrievals += 1
        
        # Close progress bar
        progress_bar.close()
    
    # Write updated data
    output_file = f"{os.path.splitext(file_path)[0]}_fetchContentAbstract.json"
    with open(output_file, 'w') as f:
        json.dump(processed_data, f, indent=4)
    
    # Log detailed retrieval statistics
    logger.info(f"Retrieval Statistics for {file_path}:")
    logger.info(f"Total Papers: {len(data)}")
    logger.info(f"Successful Retrievals: {successful_retrievals}")
    logger.info(f"Failed Retrievals: {failed_retrievals}")
    logger.info(f"Success Rate: {successful_retrievals/len(data)*100:.2f}%")
    
    print(f"Completed processing: {output_file}")
    return output_file

In [7]:
def main(input_dir, api_key):
    start_time = time.time()
    
    files = [f"{input_dir}/{file}" for file in os.listdir(input_dir) if file.endswith(".json")]
    
    for file in files:
        print(f"Processing {file}")
        process_file(file, api_key)
    
    end_time = time.time()
    print(f"Total processing time: {end_time - start_time:.2f} seconds")

In [ ]:
destination_base = "JsonFile_Scrape"
YEAR = ["2015","2016","2017","2018","2019","2020","2021","2022","2023"]
MAP_APIKEY = {"2015":"",
              "2016":"",
              "2017":"",
              "2018":"",
              "2019":"",
              "2020":"",
              "2021":"",
              "2022":"",
              "2023":""}

for year in YEAR:
    input_dir = f"{destination_base}/{year}"
    API_KEY = MAP_APIKEY[year]
    main(input_dir, API_KEY)

2024-12-05 07:08:22,476 - INFO: Processing 1250 papers from JsonFile_Scrape/2015/2015_conference_paper_not_proceeding.json


Processing JsonFile_Scrape/2015/2015_conference_paper_not_proceeding.json


Fetching Abstracts: 100%|██████████| 1250/1250 [03:07<00:00,  6.68paper/s]
2024-12-05 07:11:29,769 - INFO: Retrieval Statistics for JsonFile_Scrape/2015/2015_conference_paper_not_proceeding.json:
2024-12-05 07:11:29,770 - INFO: Total Papers: 1250
2024-12-05 07:11:29,771 - INFO: Successful Retrievals: 1250
2024-12-05 07:11:29,772 - INFO: Failed Retrievals: 0
2024-12-05 07:11:29,774 - INFO: Success Rate: 100.00%
2024-12-05 07:11:29,841 - INFO: Processing 1250 papers from JsonFile_Scrape/2015/2015_conference_paper_proceeding.json


Completed processing: JsonFile_Scrape/2015/2015_conference_paper_not_proceeding_fetchContentAbstract.json
Processing JsonFile_Scrape/2015/2015_conference_paper_proceeding.json


Fetching Abstracts: 100%|██████████| 1250/1250 [03:04<00:00,  6.79paper/s]
2024-12-05 07:14:34,243 - INFO: Retrieval Statistics for JsonFile_Scrape/2015/2015_conference_paper_proceeding.json:
2024-12-05 07:14:34,244 - INFO: Total Papers: 1250
2024-12-05 07:14:34,245 - INFO: Successful Retrievals: 1250
2024-12-05 07:14:34,247 - INFO: Failed Retrievals: 0
2024-12-05 07:14:34,248 - INFO: Success Rate: 100.00%
2024-12-05 07:14:34,307 - INFO: Processing 1250 papers from JsonFile_Scrape/2015/2015_not_conference_paper_not_proceeding.json


...

Completed processing: JsonFile_Scrape/2023/2023_conference_paper_not_proceeding_fetchContentAbstract.json
Processing JsonFile_Scrape/2023/2023_conference_paper_proceeding.json


Fetching Abstracts: 100%|██████████| 1143/1143 [02:56<00:00,  6.47paper/s]
2024-12-05 08:33:46,428 - INFO: Retrieval Statistics for JsonFile_Scrape/2023/2023_conference_paper_proceeding.json:
2024-12-05 08:33:46,430 - INFO: Total Papers: 1143
2024-12-05 08:33:46,431 - INFO: Successful Retrievals: 1143
2024-12-05 08:33:46,432 - INFO: Failed Retrievals: 0
2024-12-05 08:33:46,433 - INFO: Success Rate: 100.00%
2024-12-05 08:33:46,502 - INFO: Processing 1246 papers from JsonFile_Scrape/2023/2023_not_conference_paper_proceeding.json


Completed processing: JsonFile_Scrape/2023/2023_conference_paper_proceeding_fetchContentAbstract.json
Processing JsonFile_Scrape/2023/2023_not_conference_paper_proceeding.json


Fetching Abstracts: 100%|██████████| 1246/1246 [03:00<00:00,  6.89paper/s]
2024-12-05 08:36:47,519 - INFO: Retrieval Statistics for JsonFile_Scrape/2023/2023_not_conference_paper_proceeding.json:
2024-12-05 08:36:47,521 - INFO: Total Papers: 1246
2024-12-05 08:36:47,522 - INFO: Successful Retrievals: 1246
2024-12-05 08:36:47,524 - INFO: Failed Retrievals: 0
2024-12-05 08:36:47,525 - INFO: Success Rate: 100.00%


Completed processing: JsonFile_Scrape/2023/2023_not_conference_paper_proceeding_fetchContentAbstract.json
Total processing time: 550.65 seconds
